# Modelling NIQS Hardware TF

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import torch.optim as optim
import tensorflow as tf

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *

In [2]:
n = 2
d = 2**n
rank = 4
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

kraus_target = KrausMap(
#                 U = None,
#                 c = None,
                 d = d,
                 rank = rank,
                 trainable = False,
                 )

In [22]:
N = 540
state_index, observ_index = index_generator(n, N, trace=True)

input_list = []
circuit_list = []
for i, j in zip(state_index, observ_index):

    config = numberToBase(i, 6, n)
    state = tf.cast(prepare_input(config), dtype=tf.complex64)

    config = numberToBase(j, 4, n)
    observable = tf.cast(pauli_observable(config), dtype=tf.complex64)
    
    input_list.append([state, observable])
    
target_list = [measurement(kraus_target.apply_map(input[0])) input in input_list]
input_train, input_test = input_list[:440], input_list[440:]
target_train, target_test = target_list[:440], target_list[440:] 
optimizer = tf.optimizers.Adam(learning_rate=0.025)

In [23]:
kraus_model = KrausMap(
#                 U = None,
#                 c = None,
                 d = d,
                 rank = rank,
                 trainable = True,
                 )

model = ModelQuantumMap(q_map = kraus_model,
                        loss = expectation_value_loss,
                        input_list = input_list,
                        target_list = target_list,
                        input_val_list = input_list,
                        target_val_list = target_list,
                        optimizer = optimizer,
                        )

In [ ]:
model.train(num_iter = 10000, N = 100)

  0%|          | 0/10000 [00:00<?, ?it/s]

0 0.13023476
1 0.12649748
2 0.12242707
3 0.11849721
4 0.11443078
5 0.11049978
6 0.10661331
7 0.10287189
8 0.099169314
9 0.09561341
10 0.092094526
11 0.0887929
12 0.08552272
13 0.08252366
14 0.0797007
15 0.07694798
16 0.07439356
17 0.07198018
18 0.06973245
19 0.0676246
20 0.06555672
21 0.06360224
22 0.061735272
23 0.05991928


In [ ]:
def apply_map(state, kraus_list):
    state = [K@state@tf.linalg.adjoint(K) for K in kraus_list]
    state = tf.math.reduce_sum(tf.stack(state), axis=0)
    return state


def expectation_value(state, observable):
    ev = tf.linalg.trace(observable@state)
    return ev

In [ ]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = 8

A = tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64)
B = tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64)
G = A + 1j*B
Q, R = tf.linalg.qr(G, full_matrices = False)
D = tf.linalg.tensor_diag_part(R)
D = tf.math.sign(D)
D = tf.linalg.diag(D)
U = Q@D


kraus_target_list =  [U[i*d:(i+1)*d, :d] for i in range(rank)]

In [ ]:
N = 100
state_index, observ_index = index_generator(n, N, trace=False)

input_list = []
circuit_list = []
for i, j in zip(state_index, observ_index):

    config = numberToBase(i, 6, n)
    state = tf.cast(prepare_input(config), dtype=tf.complex64)

    config = numberToBase(j, 3, n)
    observable = tf.cast(pauli_observable(config), dtype=tf.complex64)
    
    input_list.append([state, observable])

target_list = [expectation_value(apply_map(input[0], kraus_target_list), input[1]) for input in input_list]
optimizer = tf.optimizers.Adam(learning_rate=0.05)

In [ ]:
A = tf.Variable(tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64), trainable=True)
B = tf.Variable(tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64), trainable=True)

for i in range(1000):
    with tf.GradientTape() as tape:
        G = A + 1j*B
        Q, R = tf.linalg.qr(G, full_matrices = False)
        D = tf.linalg.tensor_diag_part(R)
        D = tf.math.sign(D)
        D = tf.linalg.diag(D)
        U = Q@D

        kraus_model_list =  [U[i*d:(i+1)*d, :d] for i in range(rank)]
        pred_list = [expectation_value(apply_map(input[0], kraus_model_list), input[1]) for input in input_list]
        loss = tf.math.reduce_mean(tf.stack([(target - predicted)**2 for target, predicted in zip(target_list, pred_list)]))

    grads = tape.gradient(loss, [A, B])
    optimizer.apply_gradients(zip(grads, [A, B]))
    
    print(f"loss: {loss: .4f}, {tf.math.reduce_mean(tf.math.abs(G))}")

In [ ]:
np.random.seed(43)
A = np.random.normal(0,1, (2,2))
B = np.random.normal(0,1, (2,2))
C = np.random.normal(0,1, (2,2))

A = A@A.T
B = B@B.T
C = C@C.T

D = A + B + C
d = np.linalg.inv(sqrtm(D))
A = d@A@d
B = d@B@d
C = d@C@d

In [ ]:
print(A)
print(B)
print(C)
print(A + B + C)
print(np.linalg.eig(A)[0])
print(np.linalg.eig(B)[0])
print(np.linalg.eig(C)[0])